In [87]:
import json
import random
from pathlib import Path
import yaml
import shutil

In [88]:
RAW_DATA_JSON = "dataset/raw_blender/bboxes.json"
RAW_DATA_IMAGES = "dataset/raw_blender/results"
LABELS2ID = {'circle': 0, 'square': 1, 'triangle': 2, 'hoop': 3, 'gate': 4, 'Target circle': 5}
NUM_CLASSES = len(LABELS2ID)
IMAGE_WIDTH = 640
IMAGE_HEIGHT = 480
TEST_SIZE = 0.05
VAL_SIZE = 0.15

annotations = json.load(open(RAW_DATA_JSON))

In [89]:
len(annotations['ImageBboxes'])

10000

In [90]:
annotations['ImageBboxes'][0]

{'image_name': 'image_0.jpg',
 'bboxes': [{'x': 201,
   'y': 287,
   'width': 81,
   'height': 8,
   'color': 'purple',
   'type': 'triangle'},
  {'x': 626,
   'y': 377,
   'width': 14,
   'height': 103,
   'color': 'black',
   'type': 'hoop'},
  {'x': 0,
   'y': 293,
   'width': 79,
   'height': 12,
   'color': 'green',
   'type': 'square'},
  {'x': 426,
   'y': 325,
   'width': 101,
   'height': 13,
   'color': 'red',
   'type': 'Target circle'}]}

In [91]:
def to_yolo(annotations):
    yolo_annotations = []
    for image in annotations:
        image_annotations = []
        image_name = image['image_name']
        for bbox in image['bboxes']:
            label = bbox['type']
            x, y, w, h = bbox['x'], bbox['y'], bbox['width'], bbox['height']
            center_x, center_y = (x + w / 2) / IMAGE_WIDTH, (y + h / 2) / IMAGE_HEIGHT
            image_annotations.append([LABELS2ID[label], center_x, center_y, w / IMAGE_WIDTH, h / IMAGE_HEIGHT])
        yolo_annotations.append([image_name, image_annotations])
    return yolo_annotations

In [92]:
yolo_annotations = to_yolo(annotations['ImageBboxes'])

In [93]:
len(yolo_annotations)

10000

In [94]:
yolo_annotations[0]

['image_0.jpg',
 [[2, 0.37734375, 0.60625, 0.1265625, 0.016666666666666666],
  [3, 0.9890625, 0.8927083333333333, 0.021875, 0.21458333333333332],
  [1, 0.06171875, 0.6229166666666667, 0.1234375, 0.025],
  [5, 0.74453125, 0.690625, 0.1578125, 0.027083333333333334]]]

In [95]:
def split_annotations(annotations, test_size=TEST_SIZE, val_size=VAL_SIZE):
    random.shuffle(yolo_annotations)
    test_annotations = yolo_annotations[:int(len(yolo_annotations) * test_size)]
    val_annotations = yolo_annotations[int(
        len(yolo_annotations) * test_size):int(len(yolo_annotations) * (val_size + test_size))]
    train_annotations = yolo_annotations[int(len(yolo_annotations) * (val_size + test_size)):]
    return train_annotations, val_annotations, test_annotations

In [96]:
train_annotations, val_annotations, test_annotations = split_annotations(yolo_annotations)

In [97]:
len(train_annotations), len(val_annotations), len(test_annotations)

(8000, 1500, 500)

In [98]:
def save_yolo(annotations, images_folder, output_folder, dataset_name, subset):
    images_folder = Path(images_folder)
    output_folder = Path(output_folder) / dataset_name / subset
    output_folder.mkdir(parents=True, exist_ok=True)
    out_images_folder = output_folder / "images"
    out_images_folder.mkdir(parents=True, exist_ok=True)
    out_labels_folder = output_folder / "labels"
    out_labels_folder.mkdir(parents=True, exist_ok=True)
    for image_name, bboxes in annotations:
        shutil.copy(images_folder / image_name, out_images_folder / image_name)
        with open(out_labels_folder / (image_name.replace(".jpg", ".txt")), "w") as f:
            for bbox in bboxes:
                f.write(" ".join([str(x) for x in bbox]) + "\n")

In [99]:
# train: ../train/
# val: ../val/
# test: ../test/
# 
# nc: NUM_CLASSES
# names: 

def create_yolo_yaml(train_path, val_path, test_path, num_classes, labels, save_path):
    save_path = Path(save_path)
    save_path.mkdir(parents=True, exist_ok=True)
    out_yaml_path = save_path / "data.yaml"
    yolo_meta = {'train': train_path, 'val': val_path, 'test': test_path, 'nc': num_classes, 'names': labels}
    with open(out_yaml_path, 'w') as yaml_file:
        yaml.dump(yolo_meta, yaml_file)


In [100]:
save_yolo(train_annotations, RAW_DATA_IMAGES, "dataset", "blender", "train")
save_yolo(val_annotations, RAW_DATA_IMAGES, "dataset", "blender", "val")
save_yolo(test_annotations, RAW_DATA_IMAGES, "dataset", "blender", "test")

In [101]:
create_yolo_yaml("dataset/blender/train/", "dataset/blender/val/", "dataset/blender/test", NUM_CLASSES, list(LABELS2ID.keys()), "dataset/blender")